# 1. Crawling Starbuck list

- 서울시 내의 스타벅스 목록과 각 매장의 정보를 크롤링해 excel 형태로 저장한다.

In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By

import pandas as pd
import numpy as np

In [13]:
ser = Service('../chromedriver/chromedriver.exe')
driver = webdriver.Chrome(service=ser)
url = 'https://www.starbucks.co.kr/store/store_map.do'
driver.get(url)

In [14]:
seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, seoul_btn).click()

In [15]:
all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, all_btn).click()

In [16]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
starbucks_soup_list = soup.select('li.quickResultLstCon')[10:]
print(len(starbucks_soup_list))

568


In [17]:
starbucks_store = starbucks_soup_list[0]
starbucks_store

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

In [20]:
name = starbucks_store.select('strong')[0].text.strip()
lat = starbucks_store['data-lat'].strip()
lon = starbucks_store['data-long'].strip()
store_type = starbucks_store.select('i')[0]['class'][0][4:]
address = str(starbucks_store.select('p')[0]).split('<br/>')[0].split('>')[1]
tel = str(starbucks_store.select('p')[0]).split('<br/>')[1].split('<')[0]

In [21]:
# 전체 매장 정보 출력
starbucks_list = []

for starbucks_store in starbucks_soup_list:
    name = starbucks_store.select('strong')[0].text.strip()
    lat = starbucks_store['data-lat'].strip()
    lon = starbucks_store['data-long'].strip()
    store_type = starbucks_store.select('i')[0]['class'][0][4:]
    address = str(starbucks_store.select('p')[0]).split('<br/>')[0].split('>')[1]
    tel = str(starbucks_store.select('p')[0]).split('<br/>')[1].split('<')[0]
    
    mylist = [name, lat, lon, store_type, address, tel]
    starbucks_list.append(mylist)

In [22]:
columns = ['매장명','위도','경도','매장타입','주소','전화번호']
seoul_starbucks_df = pd.DataFrame(starbucks_list, columns=columns)
seoul_starbucks_df.head()

,매장명,위도,경도,매장타입,주소,전화번호
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.514132,127.020563,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,대치재경빌딩R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232


In [23]:
seoul_starbucks_df.to_excel('./files/seoul_starbucks_list_class.xlsx', index=False)

# 2. 서울시 Open API를 통한 자료확보

link : https://data.seoul.go.kr/

- 시군구 목록, 주민등록 인구통계, 사업체 수 데이터 등 크롤링 (시군구 목록 크롤링 생략)
- API 요청 인자 : `'http://openAPI.seoul.go.kr:8088/{KEY}/{TYPE}/{SERVICE}/{START_INDEX}/{END_INDEX}/{OBJECTID}'`
    - KEY : 인증키
    - TYPE : 요청 파일 타입(xml, xmlf, xls, json).
    - SERVICE : API의 서비스명
    - START_INDEX : 요청 시작 위치
    - END_INDEX : 요정 종료 위치
    - OBJECTID : 순번

## 2.1 API 사용 예시

In [112]:
import requests

#test
SEOUL_API_AUTH_KEY = '6c53647841796777383756766f6768'
Open_AIP = 'GangseoListLoanCompany'
url = 'http://openAPI.seoul.go.kr:8088/{}/json/{}/1/5'.format(SEOUL_API_AUTH_KEY, Open_AIP)

In [117]:
result_dict = requests.get(url).json()
result_dict

{'GangseoListLoanCompany': {'list_total_count': 1104,
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'},
  'row': [{'BSNS_SE': '대부중개업',
    'MNG_STATE_CODE': '폐업',
    'OLD_APV_NO': '2015-서울강서-00033(대부중개업)',
    'BUP_NM': '주식회사 론마음파트너스대부중개',
    'REP_NM': '김진영',
    'UPCHE_GBN': '법인',
    'WRKR_BUP_TELNO': '02-521-1012',
    'WRKR_BUP_ADDR': '서울특별시 강서구 등촌동  696번지  세신그린코아빌딩',
    'WRKR_RD_FULL_ADDR': '서울특별시 강서구 공항대로41길 51, 세신그린코아빌딩 9층 905호 (등촌동)',
    'WRKR_BUP_POST_NO': '',
    'APV_YMD': '20210712',
    'END_APV_YMD': '20240712',
    'LAST_CLS_YMD': '20220117',
    'WRKR_ESBMNT_YMD': '20150924',
    'HOFF_YN': '본점',
    'LAST_MOD_YMD': '20220118'},
   {'BSNS_SE': '대부업',
    'MNG_STATE_CODE': '폐업',
    'OLD_APV_NO': '2020-서울강서-0033(대부업)',
    'BUP_NM': '선유대부',
    'REP_NM': '김준영',
    'UPCHE_GBN': '개인',
    'WRKR_BUP_TELNO': '',
    'WRKR_BUP_ADDR': '서울특별시 강서구 가양동  14번지 51호  토피아빌딩',
    'WRKR_RD_FULL_ADDR': '서울특별시 강서구 화곡로 429, 토피아빌딩 4층 402-10호 (가양동)',
    'WRKR_BUP_POST_NO': ''

In [127]:
# data lenght
print(result_dict['GangseoListLoanCompany']['list_total_count'])

# check error
print(result_dict['GangseoListLoanCompany']['RESULT']['CODE'])

# show list
print(result_dict['GangseoListLoanCompany']['row'][0])

1104
INFO-000
{'BSNS_SE': '대부중개업', 'MNG_STATE_CODE': '폐업', 'OLD_APV_NO': '2015-서울강서-00033(대부중개업)', 'BUP_NM': '주식회사 론마음파트너스대부중개', 'REP_NM': '김진영', 'UPCHE_GBN': '법인', 'WRKR_BUP_TELNO': '02-521-1012', 'WRKR_BUP_ADDR': '서울특별시 강서구 등촌동  696번지  세신그린코아빌딩', 'WRKR_RD_FULL_ADDR': '서울특별시 강서구 공항대로41길 51, 세신그린코아빌딩 9층 905호 (등촌동)', 'WRKR_BUP_POST_NO': '', 'APV_YMD': '20210712', 'END_APV_YMD': '20240712', 'LAST_CLS_YMD': '20220117', 'WRKR_ESBMNT_YMD': '20150924', 'HOFF_YN': '본점', 'LAST_MOD_YMD': '20220118'}


In [ ]:
# API 요청 함수 작성

def seoul_open_api_data(url, service):
    data_list = None
    
    try:
        result_dict = requests.get(url).json()
        result_data = result_dict[service]
        code = result_data['RESULT']['CODE']
        if code == 'INFO-000':
            data_lst = result_data['row']
    except:
        pass
    
    return data_list

SEOUL_API_AUTH_KEY = '6c53647841796777383756766f6768'
service = 'GangseoListLoanCompany'
url = 'http://openAPI.seoul.go.kr:8088/{}/json/{}/1/5'.format(SEOUL_API_AUTH_KEY, Open_AIP)

## 2.2 자치구별 인구자료 처리

In [130]:
# 저장데이터 사용
sgg_pop_df = pd.read_csv('./files/report.txt', header=2, sep='\t')
columns = {    '기간': 'GIGAN',
    '자치구': 'JACHIGU',
    '계': 'GYE_1',
    '계.1': 'GYE_2',
    '계.2': 'GYE_3',
    '남자': 'NAMJA_1',
    '남자.1': 'NAMJA_2',
    '남자.2': 'NAMJA_3',
    '여자': 'YEOJA_1',
    '여자.1': 'YEOJA_2',
    '여자.2': 'YEOJA_3',
    '세대': 'SEDAE',
    '세대당인구': 'SEDAEDANGINGU',
    '65세이상고령자': 'N_65SEISANGGORYEONGJA'}
sgg_pop_df.rename(columns=columns, inplace=True)
sgg_pop_df.head()

,GIGAN,JACHIGU,SEDAE,GYE_1,NAMJA_1,YEOJA_1,GYE_2,NAMJA_2,YEOJA_2,GYE_3,NAMJA_3,YEOJA_3,SEDAEDANGINGU,N_65SEISANGGORYEONGJA
0,2020.3/4,합계,"4,405,833","9,953,009","4,840,912","5,112,097","9,699,232","4,719,170","4,980,062","253,777","121,742","132,035",2.20,"1,552,356"
1,2020.3/4,종로구,"74,861","159,842","77,391","82,451","149,952","73,024","76,928","9,890","4,367","5,523",2.00,"28,396"
2,2020.3/4,중구,"63,594","135,321","66,193","69,128","125,800","61,526","64,274","9,521","4,667","4,854",1.98,"24,265"
3,2020.3/4,용산구,"112,451","244,953","119,074","125,879","229,786","110,604","119,182","15,167","8,470","6,697",2.04,"39,995"
4,2020.3/4,성동구,"136,096","302,695","147,582","155,113","295,591","144,444","151,147","7,104","3,138","3,966",2.17,"45,372"


In [132]:
sgg_pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   GIGAN                  26 non-null     object 
 1   JACHIGU                26 non-null     object 
 2   SEDAE                  26 non-null     object 
 3   GYE_1                  26 non-null     object 
 4   NAMJA_1                26 non-null     object 
 5   YEOJA_1                26 non-null     object 
 6   GYE_2                  26 non-null     object 
 7   NAMJA_2                26 non-null     object 
 8   YEOJA_2                26 non-null     object 
 9   GYE_3                  26 non-null     object 
 10  NAMJA_3                26 non-null     object 
 11  YEOJA_3                26 non-null     object 
 12  SEDAEDANGINGU          26 non-null     float64
 13  N_65SEISANGGORYEONGJA  26 non-null     object 
dtypes: float64(1), object(13)
memory usage: 3.0+ KB


In [144]:
# 합계 제거
condition = sgg_pop_df['JACHIGU'] != '합계'
sgg_pop_df_select = sgg_pop_df[condition]
sgg_pop_df_select.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 1 to 25
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   GIGAN                  25 non-null     object 
 1   JACHIGU                25 non-null     object 
 2   SEDAE                  25 non-null     object 
 3   GYE_1                  25 non-null     object 
 4   NAMJA_1                25 non-null     object 
 5   YEOJA_1                25 non-null     object 
 6   GYE_2                  25 non-null     object 
 7   NAMJA_2                25 non-null     object 
 8   YEOJA_2                25 non-null     object 
 9   GYE_3                  25 non-null     object 
 10  NAMJA_3                25 non-null     object 
 11  YEOJA_3                25 non-null     object 
 12  SEDAEDANGINGU          25 non-null     float64
 13  N_65SEISANGGORYEONGJA  25 non-null     object 
dtypes: float64(1), object(13)
memory usage: 2.9+ KB


In [175]:
# 자치구, 합계 컬럼만 선택
columns = ['JACHIGU', 'GYE_1']
sgg_pop_df_final = sgg_pop_df_select[columns]
sgg_pop_df_final.columns=['시군구명','주민등록인구']
sgg_pop_df_final.reset_index(drop=True, inplace=True)
#sgg_pop_df_final.to_excel('./files/sgg_pop.xlsx', index=False)
sgg_pop_df_final.head()

,시군구명,주민등록인구
0,종로구,"159,842"
1,중구,"135,321"
2,용산구,"244,953"
3,성동구,"302,695"
4,광진구,"361,923"


## 2.3 자치구별 사업자 자료 처리

In [152]:
sgg_biz_df = pd.read_csv('./files/report2.txt', header=2, sep='\t')
sgg_biz_df.head()

,기간,자치구,동,사업체수,여성대표자,계,남,여,사업체수.1,종사자수,...,사업체수.15,종사자수.14,사업체수.16,종사자수.15,사업체수.17,종사자수.16,사업체수.18,종사자수.17,사업체수.19,종사자수.18
0,2018,합계,합계,"823,385","279,711","5,210,936","2,878,227","2,332,709",30,462,...,"1,282","140,121","35,377","346,219","28,824","378,366","23,025","91,104","69,694","191,423"
1,2018,종로구,소계,"39,952","13,442","265,017","152,717","112,300",3,17,...,89,"15,658",850,"13,504",744,"15,888",909,"6,227","2,448","7,672"
2,2018,종로구,사직동,"3,541","1,191","49,536","28,658","20,878",1,14,...,19,"6,312",86,967,103,870,95,"1,544",252,843
3,2018,종로구,삼청동,712,324,"4,577","2,193","2,384",-,-,...,4,"1,204",14,260,7,30,17,175,24,209
4,2018,종로구,부암동,565,240,"3,609","1,706","1,903",-,-,...,3,59,40,"1,424",15,148,16,40,83,362


In [153]:
sgg_biz_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 46 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   기간       450 non-null    int64 
 1   자치구      450 non-null    object
 2   동        450 non-null    object
 3   사업체수     450 non-null    object
 4   여성대표자    450 non-null    object
 5   계        450 non-null    object
 6   남        450 non-null    object
 7   여        450 non-null    object
 8   사업체수.1   450 non-null    object
 9   종사자수     450 non-null    object
 10  사업체수.2   450 non-null    object
 11  종사자수.1   450 non-null    object
 12  사업체수.3   450 non-null    object
 13  종사자수.2   450 non-null    object
 14  사업체수.4   450 non-null    object
 15  종사자수.3   450 non-null    object
 16  사업체수.5   450 non-null    object
 17  종사자수.4   450 non-null    object
 18  사업체수.6   450 non-null    object
 19  종사자수.5   450 non-null    object
 20  사업체수.7   450 non-null    object
 21  종사자수.6   450 non-null    object
 22  사업

In [164]:
condition = sgg_biz_df['동']=='소계'
sgg_biz_df_select = sgg_biz_df[condition]

columns = ['자치구','계','사업체수']
sgg_biz_df_final = sgg_biz_df_select[columns]
sgg_biz_df_final.columns = ['시군구명','종사자수','사업체수']
sgg_biz_df_final.reset_index(drop=True, inplace=True)
sgg_biz_df_final.head()

,시군구명,종사자수,사업체수
0,종로구,"265,017","39,952"
1,중구,"390,530","60,957"
2,용산구,"141,216","20,813"
3,성동구,"174,819","27,868"
4,광진구,"127,879","24,535"


# 3. 데이터 전처리
- 지역구별 스타벅스 매장수를 시군구 목록 데이터에 추가
- 지역구별 인구수 자료를 시군구 목록 데이터에 추가
- 지역구별 사업자 자료를 시군구 목록 데이터에 추가

In [3]:
# 스타벅스 자료
seoul_starbucks = pd.read_excel('./files/seoul_starbucks_list_class.xlsx')
seoul_starbucks['시군구명'] = seoul_starbucks['주소'].map(lambda x: x.split(' ')[1])
seoul_starbucks.head()

,매장명,위도,경도,매장타입,주소,전화번호,시군구명
0,역삼아레나빌딩,37.501087,37.501087,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232,강남구
1,논현역사거리,37.510178,37.510178,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232,강남구
2,신사역성일빌딩,37.514132,37.514132,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232,강남구
3,국기원사거리,37.499517,37.499517,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,강남구
4,대치재경빌딩R,37.494668,37.494668,reserve,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,강남구


In [4]:
# 전처리 : 지역구별 스타벅스 매장 수
starbucks_ssg_count = seoul_starbucks.pivot_table(index='시군구명', values='매장명',
                                                 aggfunc='count')
starbucks_ssg_count.reset_index(inplace=True)
starbucks_ssg_count.head()

,시군구명,매장명
0,강남구,88
1,강동구,16
2,강북구,6
3,강서구,22
4,관악구,11


In [5]:
# 시군구 목록 데이터 병합
seoul_ssg = pd.read_excel('./files/seoul_sgg_list.xlsx')
seoul_ssg.head()

,시군구코드,시군구명,위도,경도
0,11320,도봉구,37.665861,127.031767
1,11380,은평구,37.617612,126.922700
2,11230,동대문구,37.583801,127.050700
3,11590,동작구,37.496504,126.944307
4,11545,금천구,37.460097,126.900155


In [6]:
seoul_ssg = pd.merge(left=seoul_ssg, right=starbucks_ssg_count, how='left',)
seoul_ssg.head()

,시군구코드,시군구명,위도,경도,매장명
0,11320,도봉구,37.665861,127.031767,3
1,11380,은평구,37.617612,126.922700,8
2,11230,동대문구,37.583801,127.050700,9
3,11590,동작구,37.496504,126.944307,11
4,11545,금천구,37.460097,126.900155,11


In [7]:
# 지역구별 인구수 자료 병합
seoul_ssg_pop = pd.read_excel('./files/sgg_pop.xlsx')
seoul_ssg_pop.head()

,시군구명,주민등록인구
0,종로구,"159,842"
1,중구,"135,321"
2,용산구,"244,953"
3,성동구,"302,695"
4,광진구,"361,923"


In [8]:
seoul_ssg = pd.merge(left=seoul_ssg, right=seoul_ssg_pop, how='left')
seoul_ssg.head()

,시군구코드,시군구명,위도,경도,매장명,주민등록인구
0,11320,도봉구,37.665861,127.031767,3,"329,300"
1,11380,은평구,37.617612,126.922700,8,"485,842"
2,11230,동대문구,37.583801,127.050700,9,"358,679"
3,11590,동작구,37.496504,126.944307,11,"404,617"
4,11545,금천구,37.460097,126.900155,11,"249,641"


In [9]:
# 지역구별 사업자 자료 병합
seoul_ssg_biz = pd.read_excel('./files/sgg_biz.xlsx')
seoul_ssg_biz.head()

,시군구명,종사자수,사업체수
0,종로구,"265,017","39,952"
1,중구,"390,530","60,957"
2,용산구,"141,216","20,813"
3,성동구,"174,819","27,868"
4,광진구,"127,879","24,535"


In [10]:
seoul_ssg = pd.merge(left=seoul_ssg, right=seoul_ssg_biz, how='left')
seoul_ssg.head()

,시군구코드,시군구명,위도,경도,매장명,주민등록인구,종사자수,사업체수
0,11320,도봉구,37.665861,127.031767,3,"329,300","74,314","18,894"
1,11380,은평구,37.617612,126.922700,8,"485,842","90,049","24,873"
2,11230,동대문구,37.583801,127.050700,9,"358,679","140,748","31,878"
3,11590,동작구,37.496504,126.944307,11,"404,617","112,564","19,916"
4,11545,금천구,37.460097,126.900155,11,"249,641","234,012","32,472"


In [11]:
seoul_ssg.to_excel('./files/seoul_sgg_stat_class.xlsx', index=False)